# Notebook com as funções necessárias para obter as informações de títulos públicos

## Definições dos imports

In [1]:
# imports de bibliotecas
import pandas as pd
import pdb


## Definições de variáveis globais

In [2]:
# Definições de URLs

## URL dados do Tesouro Direto
url_CSV_TesouroDireto = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'


## Função para recuperar dados do Tesouro Transparente

In [3]:
def buscar_titulos_tesouro_direto(url):
  df = pd.read_csv(url, sep=';', decimal=',')
  df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
  df['Data Base'] = pd.to_datetime(df['Data Base'], dayfirst=True)
  df.sort_values(by=['Data Base', 'Data Vencimento'], inplace=True)
  return df

## Função para obter os retornos dos títulos públicos

In [ ]:
def buscarRetornosTitulosPublicos(array_df_tipoativos,data_inicio, data_fim):
  colecao_retornos_calculados = []
  # O valor atribuído à variável é para referenciar a posição no array correspondente a títulos públicos
  i = 2
  titulos = array_df_tipoativos[i].iloc[:,0]
  titulos.to_numpy()

   # Lendo a base de títulos públicos
  dfDadosTitulosPublicos = buscar_titulos_tesouro_direto(url_CSV_TesouroDireto)

  dfDadosTitulosPublicos_alterado = dfDadosTitulosPublicos
  dfDadosTitulosPublicos_alterado['Descricao'] = dfDadosTitulosPublicos['Tipo Titulo'].astype(str) + ' ' + dfDadosTitulosPublicos['Data Vencimento'].dt.strftime("%Y")

  dfDadosTitulosPublicos_alterado = dfDadosTitulosPublicos_alterado.iloc[:,[8,2,7]]

  dfDadosTitulosPublicos_juncao = pd.DataFrame()

  for titulo in titulos:
    #pdb.set_trace()
    dfTemp = dfDadosTitulosPublicos_alterado[(dfDadosTitulosPublicos_alterado["Descricao"].str.upper() == titulo.upper()) & ((dfDadosTitulosPublicos_alterado["Data Base"] >= data_inicio) & (dfDadosTitulosPublicos_alterado["Data Base"] <= data_fim))]
    data = [dfTemp, dfDadosTitulosPublicos_juncao]
    dfDadosTitulosPublicos_juncao = pd.concat(data, sort=True)

  dfDadosTitulosPublicos_juncao = dfDadosTitulosPublicos_juncao.pivot_table(columns='Descricao',values='PU Base Manha', index='Data Base', margins=True)
  dfDadosTitulosPublicos_juncao = dfDadosTitulosPublicos_juncao[:-1]

  dfDadosTitulosPublicos_juncao.index = pd.to_datetime(dfDadosTitulosPublicos_juncao.index)

  precos_ativos = dfDadosTitulosPublicos_juncao[:].resample('Y').last()

  retornos_calculados = precos_ativos.pct_change()[1:]

  retornos_calculados.drop('All', inplace=True, axis=1)


  return retornos_calculados.round(2)


## Função para obter retornos de títulos públicos como array

In [ ]:
def buscarRetornosTitulosPublicos_from_array(array_tipoativos,data_inicio, data_fim):
  colecao_retornos_calculados = []
  # O valor atribuído à variável é para referenciar a posição no array correspondente a títulos públicos
  i = 2
  titulos = array_tipoativos
  #titulos.to_numpy()

   # Lendo a base de títulos públicos
  dfDadosTitulosPublicos = buscar_titulos_tesouro_direto(url_CSV_TesouroDireto)

  dfDadosTitulosPublicos_alterado = dfDadosTitulosPublicos
  dfDadosTitulosPublicos_alterado['Descricao'] = dfDadosTitulosPublicos['Tipo Titulo'].astype(str) + ' ' + dfDadosTitulosPublicos['Data Vencimento'].dt.strftime("%Y")

  dfDadosTitulosPublicos_alterado = dfDadosTitulosPublicos_alterado.iloc[:,[8,2,7]]

  dfDadosTitulosPublicos_juncao = pd.DataFrame()

  for titulo in titulos:
    #pdb.set_trace()
    dfTemp = dfDadosTitulosPublicos_alterado[(dfDadosTitulosPublicos_alterado["Descricao"].str.upper() == titulo.upper()) & ((dfDadosTitulosPublicos_alterado["Data Base"] >= data_inicio) & (dfDadosTitulosPublicos_alterado["Data Base"] <= data_fim))]
    data = [dfTemp, dfDadosTitulosPublicos_juncao]
    dfDadosTitulosPublicos_juncao = pd.concat(data, sort=True)

  dfDadosTitulosPublicos_juncao = dfDadosTitulosPublicos_juncao.pivot_table(columns='Descricao',values='PU Base Manha', index='Data Base', margins=True)
  dfDadosTitulosPublicos_juncao = dfDadosTitulosPublicos_juncao[:-1]

  dfDadosTitulosPublicos_juncao.index = pd.to_datetime(dfDadosTitulosPublicos_juncao.index)

  precos_ativos = dfDadosTitulosPublicos_juncao[:].resample('Y').last()

  retornos_calculados = precos_ativos.pct_change()[1:]

  retornos_calculados.drop('All', inplace=True, axis=1)


  return retornos_calculados.round(2)

## Função para retornar a cotação corrente de títulos públicos

In [ ]:
def buscarCotacaoAtivoRendaFixa(array_df_tipoativos):
  colecao_retornos_calculados = []
  # O valor atribuído à variável é para referenciar a posição no array correspondente a títulos públicos
  i = 2
  titulos = array_df_tipoativos[i].iloc[:,0]
  titulos.to_numpy()
  # Lendo a base de títulos públicos
  dfDadosTitulosPublicos = buscar_titulos_tesouro_direto(url_CSV_TesouroDireto)
  dfDadosTitulosPublicos_alterado = dfDadosTitulosPublicos
  dfDadosTitulosPublicos_alterado['Descricao'] = dfDadosTitulosPublicos['Tipo Titulo'].astype(str) + ' ' + dfDadosTitulosPublicos['Data Vencimento'].dt.strftime("%Y")
  dfDadosTitulosPublicos_alterado = dfDadosTitulosPublicos_alterado.iloc[:,[8,2,5]]

  dfDadosTitulosPublicos_juncao = pd.DataFrame()

  for titulo in titulos:
    #pdb.set_trace()
    dfTemp = dfDadosTitulosPublicos_alterado[(dfDadosTitulosPublicos_alterado["Descricao"].str.upper() == titulo.upper())]
    data = [dfTemp, dfDadosTitulosPublicos_juncao]
    dfDadosTitulosPublicos_juncao = pd.concat(data, sort=True)

  dfDadosTitulosPublicos_juncao = dfDadosTitulosPublicos_juncao.pivot_table(columns='Descricao',values='PU Compra Manha', index='Data Base', margins=True)
  dfDadosTitulosPublicos_juncao = dfDadosTitulosPublicos_juncao[:-1]

  dfDadosTitulosPublicos_juncao.index = pd.to_datetime(dfDadosTitulosPublicos_juncao.index)

  precos_ativos = dfDadosTitulosPublicos_juncao[:].resample('Y').last()

  df_precos_titulos = precos_ativos.transpose()
  df_precos_titulos = df_precos_titulos.iloc[:,-1:][:-1]
  df_precos_titulos.columns = ['valor']

  return df_precos_titulos.round(2)



# APAGAR DEPOIS

In [8]:
#df_tst = buscar_titulos_tesouro_direto(url_CSV_TesouroDireto)

In [12]:
# df_tst

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha
25245,Tesouro Prefixado,2005-01-04,2004-12-31,17.74,17.78,999.35,999.35,998.70
25239,Tesouro Selic,2005-02-16,2004-12-31,0.00,0.02,2162.80,2162.74,2161.34
25243,Tesouro Prefixado,2005-04-01,2004-12-31,18.36,18.40,960.01,959.94,959.29
25242,Tesouro Prefixado,2005-07-01,2004-12-31,18.55,18.61,919.67,919.44,918.82
25251,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2004-12-31,9.23,9.25,1776.67,1776.51,1827.87
...,...,...,...,...,...,...,...,...
37458,Tesouro Renda+ Aposentadoria Extra,2064-12-15,2023-09-22,5.80,5.92,753.39,728.32,728.32
37461,Tesouro Renda+ Aposentadoria Extra,2069-12-15,2023-09-22,5.81,5.93,567.00,545.05,545.05
37455,Tesouro Renda+ Aposentadoria Extra,2074-12-15,2023-09-22,5.82,5.94,426.29,407.49,407.49
37457,Tesouro Renda+ Aposentadoria Extra,2079-12-15,2023-09-22,5.82,5.94,321.55,305.63,305.63


In [19]:
# df_tst.loc[(df_tst['Tipo Titulo'] == 'Tesouro Prefixado') & (df_tst['Data Base'] >= '2010-12-31')]

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha
120936,Tesouro Prefixado,2011-07-01,2010-12-31,11.63,11.67,947.30,947.13,946.72
120934,Tesouro Prefixado,2012-01-01,2010-12-31,12.10,12.14,892.46,892.14,891.74
120935,Tesouro Prefixado,2013-01-01,2010-12-31,12.34,12.40,793.10,792.26,791.89
120903,Tesouro Prefixado,2011-07-01,2011-01-03,11.63,11.66,947.71,947.59,947.17
120904,Tesouro Prefixado,2012-01-01,2011-01-03,12.10,12.14,892.86,892.55,892.14
...,...,...,...,...,...,...,...,...
37483,Tesouro Prefixado,2029-01-01,2023-09-21,11.11,11.23,574.92,571.43,571.43
37431,Tesouro Prefixado,2024-07-01,2023-09-22,11.12,11.24,923.57,922.43,922.43
37432,Tesouro Prefixado,2025-01-01,2023-09-22,10.48,10.60,881.12,879.55,879.55
37433,Tesouro Prefixado,2026-01-01,2023-09-22,10.28,10.40,800.51,798.22,798.22
